In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load datasets
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Drop unnecessary columns (like 'id' and 'Name')
unnecessary_cols = [ "Name"]
train_df.drop(columns=unnecessary_cols, inplace=True, errors='ignore')
test_df.drop(columns=unnecessary_cols, inplace=True, errors='ignore')

# Identify categorical columns (assumes object-type columns are categorical)
categorical_columns = train_df.select_dtypes(include=["object"]).columns.tolist()

# Encode categorical features
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])

    # Handle unseen labels in the test set
    test_df[col] = test_df[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
    
    label_encoders[col] = le

# Separate features and target
X = train_df.drop(columns=["Depression"], axis=1)
y = train_df["Depression"]

# Scale numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_scaled = scaler.transform(test_df)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

# Predict on test data
test_predictions = model.predict(test_scaled)

# Create submission file
submission = pd.DataFrame({"id": test_df.id, "Depression": test_predictions})
submission.to_csv("submission1.csv", index=False)
print("Submission file saved as submission.csv")


Validation Accuracy: 0.9366
Submission file saved as submission.csv
